In [1]:
import numpy as np

In [2]:
class Matrix:
    def __init__(self, size):
        ## Array is defined by array[row][column]
        columns = size
        rows = size
        self.array = [[0 for _ in range(columns)] for _ in range(rows)]
        self.size = size
    
    def set_matrix(self, new_matrix):
        self.array = new_matrix
    def get_matrix(self):
        return self.array
    def set_value(self, row, column, value):
        self.array[row][column] = value
    def get_value(self, row, column):
        return self.array[row][column]
    def multiply(self, another_matrix, operator):
        other_array = another_matrix.get_matrix()
        assert len(other_array) == len(other_array[0]) == self.size 
        result = [[0 for _ in range(self.size)] for _ in range(self.size)]
        for thisRow in range(len(self.array)):
            for thisColumn in range(len(self.array[0])):
                result[thisRow][thisColumn] = self.__helper_mult(thisRow, thisColumn, other_array, operator)
        return result
    
    def __helper_mult(self, row, column, other_array, operator):
        total = operator.OP1_ID
        for c in range(len(other_array)):
            other_value = other_array[c][column]
            this_value = self.array[row][c]
            total = operator.op1(total,operator.op2(this_value, other_value))
        return total
    def copy(self):
        new_array = [row[:] for row in self.array]
        result = Matrix(size)
        result.set_matrix(new_array)
        return result
    
    def printSelf(self):
        print(np.matrix(self.array))
    
class AddMult:
    OP1_ID = 0
    OP2_ID = 1
    @staticmethod
    def op1(a, b):
        return a + b
    @staticmethod
    def op2(a, b):
        return a * b
    
def printMatrix(array):
    print(np.matrix(array))

In [3]:
m = Matrix(3)
m.set_value(0,0,1)
m.set_value(1,1,3)
m.set_value(2,2,1)
m.set_value(2,1,2)
m.printSelf()
print()
printMatrix(m.multiply(m, AddMult()))


[[1 0 0]
 [0 3 0]
 [0 2 1]]

[[1 0 0]
 [0 9 0]
 [0 8 1]]


In [4]:
class Node():
    def __init__(self, data):
        self.data = data
        self.links = []
        self.strlinks = []
        
    def addLink(self, link):
        self.links.append(link)
        self.strlinks.append(link.data)
        
    def __str__(self):
        if self.data == None:
            return "None"
        s = self.data + "\n"
        for item in self.links:
            s += "\t" + item.data + "\n"
        return s
        



In [5]:
# http://www.netinstructions.com/how-to-make-a-web-crawler-in-under-50-lines-of-python-code/

from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse




class Crawler():
    
    def __init__(self, root):
        self.front = Node(root)
        self.current = None
        # this is actually a queue im sorry
        self.stack = []
        self.depth = 0
        self.numFound = 0
        self.visitedLinks = []
        
    def addNode(self, n):
        
        if n.data not in self.visitedLinks:
            self.current.addLink(Node(n.data))
            self.stack.append(self.current.links[-1])
            self.visitedLinks.append(n.data)
        elif n.data not in self.current.strlinks:
            self.current.addLink(n)

        
    def crawl(self, maxPages):
        numberVisited = 0
        self.stack.append(self.front)
        # The main loop. Create a LinkParser and get all the links on the page.
        # Also search the page for the word or string
        # In our getLinks function we return the web page
        # (this is useful for searching for the word)
        # and we return a set of links from that web page
        # (this is useful for where to go next)

        while not len(self.stack) == 0 and len(self.visitedLinks) < maxPages:
            # Start from the beginning of our collection of pages to visit:
            n = self.stack.pop(0)
            try:
                self.current = n
                numberVisited += 1
                #print(numberVisited, "Visiting:", node.data)
                parser = LinkParser(self)
                data = parser.getLinks(n.data)
            except:
                print(self.current.data + " Error")
                if len(self.stack) != 0:
                    self.current = self.stack[-1]
            if len(self.visitedLinks) % 500 == 0 :
                print("have visited: " + str(len(self.visitedLinks)))
        return self.front
        

         


# We are going to create a class called LinkParser that inherits some
# methods from HTMLParser which is why it is passed into the definition
class LinkParser(HTMLParser):
    
    def __init__(self, crawler):
        HTMLParser.__init__(self)
        self.crawler = crawler
        

    # This is a function that HTMLParser normally has
    # but we are adding some functionality to it
    def handle_starttag(self, tag, attrs):
        # We are looking for the begining of a link. Links normally look
        # like <a href="www.someurl.com"></a>
        if tag == 'a':
            for (key, value) in attrs:

                if key == 'href':
                    # We are grabbing the new URL. We are also adding the
                    # base URL to it. For example:
                    # www.netinstructions.com is the base and
                    # somepage.html is the new URL (a relative URL)
                    #
                    # We combine a relative URL with the base URL to create
                    # an absolute URL like:
                    # www.netinstructions.com/somepage.html
                    newUrl = parse.urljoin(self.baseUrl, value)
                    newUrl = newUrl.split("#")[0]
                    if "www.hendrix" in newUrl:
                        self.crawler.numFound += 1
                        newLink = Node(newUrl)
                        self.crawler.addNode(newLink)

                    # And add it to our colection of links:

                    #self.links = self.links + [newUrl]


                    

    # This is a new function that we are creating to get links
    # that our spider() function will call
    def getLinks(self, url):
        
        # Remember the base URL which will be important when creating
        # absolute URLs
        self.baseUrl = url
        # Use the urlopen function from the standard Python 3 library
        response = urlopen(url)
        # Make sure that we are looking at HTML and not other things that
        # are floating around on the internet (such as
        # JavaScript files, CSS, or .PDFs for example)
        if 'text/html' in response.getheader('Content-Type'):
            htmlBytes = response.read()
            # Note that feed() handles Strings well, but not bytes
            # (A change from Python 2.x to Python 3.x)
            htmlString = htmlBytes.decode("utf-8")
            self.feed(htmlString)
            
            #print(self.links)
            return htmlString
        else:
            return "",[]
        
c = Crawler("https://www.hendrix.edu/")
result = c.crawl(10000)

https://www.hendrix.edu/courses/ Error
https://www.hendrix.edu/discoverhendrix/fastfacts/ Error
https://www.hendrix.edu/WorkArea/DownloadAsset.aspx?id=82491 Error
http://www.hendrixwarriors.com Error
https://www.hendrix.edu/webmail Error
https://www.hendrix.edu/uploadedImages/Departments_and_Programs/Art/Printmaking 02.jpg?width=820&height=360&mode=crop Error
https://www.hendrix.edu/uploadedImages/Departments_and_Programs/Art/Photography 03.jpg?width=820&height=360&mode=crop Error
https://www.hendrix.edu/uploadedImages/Departments_and_Programs/Art/Gallery/Painting 01.jpeg?width=820&height=360&mode=crop Error
https://www.hendrix.edu/art/scholarships/ Error
https://www.hendrix.edu/assetmanagement/DownloadAsset.aspx?ID=e25fac70-a7aa-4407-bf3f-313c6feb073f&version=37402e6fd71647aabb57f6b7d75acbe61.pdf Error
https://www.hendrix.edu/uploadedFiles/Departments_and_Programs/Environmental_Studies/symposium II.pdf Error
https://www.hendrix.edu/assetmanagement/DownloadAsset.aspx?ID=237cb047-f2e3-4

TypeError: must be str, not int

In [12]:
for item in result.links:
    print(item)


https://www.hendrix.edu/life/
	https://www.hendrix.edu/life/
	https://www.hendrix.edu/academics/
	https://www.hendrix.edu/visit/
	https://www.hendrix.edu/financialaid/
	https://www.hendrix.edu/admission/
	https://www.hendrix.edu/apply
	https://www.hendrix.edu/currentstudents/
	https://www.hendrix.edu/parents/
	https://www.hendrix.edu/alumni/
	https://www.hendrix.edu/facultystaff/
	https://www.hendrix.edu/CampusWeb
	https://www.hendrix.edu/news
	https://www.hendrix.edu/giving
	https://www.hendrix.edu/
	https://www.hendrix.edu/Pages/Templates/2015-PageBuilder.aspx?pageid=75396
	https://www.hendrix.edu/news/news.aspx?id=76074
	https://www.hendrix.edu/life/studentactivities/
	https://www.hendrix.edu/life/diningservices/
	https://www.hendrix.edu/life/LivingOnCampus/
	https://www.hendrix.edu/life/livingoncampus/
	https://www.hendrix.edu/life/athletics/
	https://www.hendrix.edu/life/religiouslife/
	https://www.hendrix.edu/life/wellness/
	https://www.hendrix.edu/abouthendrix/
	https://www.hend

In [10]:
c.visitedLinks

['https://www.hendrix.edu/life/',
 'https://www.hendrix.edu/academics/',
 'https://www.hendrix.edu/visit/',
 'https://www.hendrix.edu/financialaid/',
 'https://www.hendrix.edu/admission/',
 'https://www.hendrix.edu/apply',
 'https://www.hendrix.edu/currentstudents/',
 'https://www.hendrix.edu/parents/',
 'https://www.hendrix.edu/alumni/',
 'https://www.hendrix.edu/facultystaff/',
 'https://www.hendrix.edu/',
 'https://www.hendrix.edu/CampusWeb',
 'https://www.hendrix.edu/news',
 'https://www.hendrix.edu/giving',
 'https://www.hendrix.edu/#RequestInfo',
 'https://www.hendrix.edu/academics/odyssey/',
 'https://www.hendrix.edu/economics/',
 'https://www.hendrix.edu/art/',
 'https://www.hendrix.edu/bcmb/',
 'https://www.hendrix.edu/biology/',
 'https://www.hendrix.edu/chemicalphysics/',
 'https://www.hendrix.edu/chemistry/',
 'https://www.hendrix.edu/classics/',
 'https://www.hendrix.edu/mathcs/',
 'https://www.hendrix.edu/english/',
 'https://www.hendrix.edu/environmentalstudies/',
 'http

In [6]:
import queue

In [7]:
def tree_to_matrix(Treeroot, crawler):
    mat = Matrix(len(crawler.visitedLinks))   
    ## Assign each link to a index
    current_index = 0
    index_to_link = {}
    link_to_index = {}
    for link in crawler.visitedLinks:
        index_to_link[current_index] = link
        link_to_index[link] = current_index
        current_index += 1
    ## Do BFS 
    q = queue.Queue()
    q.put(Treeroot)
    seenURLs = []
    while not q.empty():
        current_node = q.get()
        current_index = link_to_index[current_node.data]
        for node in current_node.links:
            if node.data not in seenURLs:
                seenURLs.append(node.data)
                q.put(node)
            link_end_index = link_to_index[node.data]
            mat.set_value(current_index, link_end_index, mat.get_value(current_index, link_end_index) + 1)
            mat.set_value(link_end_index,current_index, mat.get_value(link_end_index, current_index) + 1)
    return (mat,index_to_link,link_to_index)


In [8]:
resulting_matrix , index_to_link_dict, link_to_index_dict = tree_to_matrix(result, c)

In [9]:
resulting_matrix.printSelf()

[[2 2 2 ..., 0 0 0]
 [2 2 2 ..., 0 0 0]
 [2 2 2 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [10]:
np.savetxt("result1000.csv", np.array(resulting_matrix.array), fmt='%i', delimiter=",")

In [34]:
fullprint(resulting_matrix)

In [12]:
index_to_link_dict[159]

'https://www.hendrix.edu/courses/'

In [57]:
link_to_index_dict["https://www.hendrix.edu/"]

10